## Doing
 - [ ] 7 day rolling averages
## To Do
 - [ ] pull data from JHU
 - [ ] pull data directly from usafacts 
 - [ ] Allow run in COLAB and save to Google Drive

## Done
 - [x] ensure proper data types in df_combined_tidy
 - [x] make tidy data one row per observation (observations are by date)
 - [x] rename variable column 'Date' in df_combined_tidy


In [1]:
import pandas as pd
import urllib3 as urllib
import json
import math
from datetime import datetime

import IPython.display
import altair as alt

pd.options.display.max_columns = None

In [2]:
jhu_confirmed_df= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

# PICK ONE AND COMMENT OUT THE OTHER (local or GIT)

# READ from Local Files
# usafacts_confirmed_df= pd.read_csv('County-Data/covid_confirmed_usafacts.csv')
# usafacts_population_df= pd.read_csv('County-Data/covid_county_population_usafacts.csv')

# READ from GIT Files
usafacts_confirmed_df= pd.read_csv('https://github.com/JaviThomas/Covid-County-LTC-Analysis/blob/master/County-Analysis/County-Data/covid_confirmed_usafacts.csv?raw=true')
usafacts_population_df= pd.read_csv('https://github.com/JaviThomas/Covid-County-LTC-Analysis/blob/master/County-Analysis/County-Data/covid_county_population_usafacts.csv?raw=true')

In [3]:
# usafacts_population_df['State_County_FIPS'] = usafacts_population_df['State'] +  usafacts_population_df['County Name'] +  usafacts_population_df['countyFIPS'].astype('str')
# usafacts_confirmed_df['State_County_FIPS'] = usafacts_confirmed_df['State'] +  usafacts_confirmed_df['County Name'] +  usafacts_confirmed_df['countyFIPS'].astype('str')

In [4]:
df_confirmed = usafacts_confirmed_df[usafacts_confirmed_df['countyFIPS'] > 2].copy()
df_population = usafacts_population_df[usafacts_population_df['countyFIPS'] > 2].copy()

df_population.set_index('countyFIPS', inplace=True)
df_confirmed.set_index('countyFIPS', inplace=True)

df_confirmed_combined= pd.concat ([df_confirmed, df_population['population']], axis=1).copy()
df_confirmed_combined.reset_index(inplace=True)
# REMOVE "Counties" where population is ZERO
df_cases_clean = df_confirmed_combined[df_confirmed_combined.population != 0]
df_cases_clean.head(5)

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,population
0,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,7,7,10,10,12,12,12,12,12,12,17,17,19,19,19,23,25,25,25,25,27,28,30,32,33,36,37,37,39,40,42,42,42,45,48,53,53,58,61,67,68,74,84,91,93,103,103,110,110,120,127,136,147,149,155,159,173,189,192,205,212,216,220,233,238,239,241,248,259,265,272,282,295,312,323,331,357,368,373,375,400,411,431,434,442,453,469,479,488,498,503,527,537,553,561,568,591,615,618,644,651,661,670,684,706,728,746,756,780,789,827,842,857,865,886,905,921,932,942,965,974,974,1002,1015,1030,1052,1066,1073,1073,1096,1113,1134,1215,1215,1215,1241,1250,1252,1262,1273,1274,1291,1293,1293,1293,1322,1324,1351,1355,1366,1377,1389,1400,1438,1442,1452,1452,1466,1475,1492,1498,1504,1508,1522,1544,1551,1565,1576,1585,1601,1619,1624,1664,1673,1690,1691,1714,1715,1715,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911,1924,1928,1949,1966,1983,1989,1999

In [5]:
# date_cols = list(df_combined_clean.columns[3:-2])
id_cols = ['County Name', 'State', 'countyFIPS', 'stateFIPS', 'population']

df_cases_tidy = df_cases_clean.melt(id_vars =id_cols, value_vars = df_cases_clean.columns[4:-2], var_name='Date', value_name='Cases')

df_cases_tidy['Date'] = df_cases_tidy['Date'].astype(str)
df_cases_tidy['Date'] = pd.to_datetime(df_cases_tidy['Date'], format='%m/%d/%y')

display(df_cases_tidy.dtypes)
df_cases_tidy

County Name            object
State                  object
countyFIPS              int64
stateFIPS               int64
population              int64
Date           datetime64[ns]
Cases                   int64
dtype: object

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases
0,Autauga County,AL,1001,1,55869,2020-01-22,0
1,Baldwin County,AL,1003,1,223234,2020-01-22,0
2,Barbour County,AL,1005,1,24686,2020-01-22,0
3,Bibb County,AL,1007,1,22394,2020-01-22,0
4,Blount County,AL,1009,1,57826,2020-01-22,0
...,...,...,...,...,...,...,...
1156251,Sweetwater County,WY,56037,56,42343,2021-01-23,3384
1156252,Teton County,WY,56039,56,23464,2021-01-23,2870
1156253,Uinta County,WY,56041,56,20226,2021-01-23,1887
1156254,Washakie County,WY,56043,56,7805,2021-01-23,853


In [6]:
#Save to CSV file
# df_cases_tidy.to_csv('County-Data/usafacts_cases_tidy.csv')

In [7]:
df_cases_tidy.sort_values(by=['countyFIPS', 'Date'], inplace=True)

# Step 2 - Get Rolling Averages

In [8]:
%%time
first_time = True
pd.set_option('mode.chained_assignment', None)
state = 'FL'
mydf = df_cases_tidy[(df_cases_tidy['State'] == state)]
for countyFIPS in mydf['countyFIPS'].unique():
#for countyFIPS in df_cases_tidy['countyFIPS'].unique():

    tmp_df = df_cases_tidy[df_cases_tidy['countyFIPS'] == countyFIPS]
#     tmp_df['7day'] = tmp_df.rolling(7).mean()['Cases']
    tmp_df['NewCases'] = tmp_df['Cases'] - tmp_df['Cases'].shift(1)
    tmp_df['7day_avg'] = tmp_df['NewCases'].rolling(7).mean().round(1)
    tmp_df['14day_avg'] = tmp_df['NewCases'].rolling(14).mean().round(1)
    tmp_df['28day_avg'] = tmp_df['NewCases'].rolling(28).mean().round(1)


    if first_time:
        df_cases_7day_tidy = tmp_df.copy()
        first_time = False
        print('First')
    else:
        df_cases_7day_tidy = df_cases_7day_tidy.append(tmp_df)

pd.set_option('mode.chained_assignment', 'warn')

First
Wall time: 1.28 s


In [9]:
print(len(df_cases_7day_tidy))
len(df_cases_tidy)

24656


1156256

In [10]:
df_cases_7day_tidy.tail(5)

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg
1140932,Washington County,FL,12133,12,25473,2021-01-19,2282,9.0,14.7,16.1,17.0
1144074,Washington County,FL,12133,12,25473,2021-01-20,2295,13.0,14.0,14.9,16.6
1147216,Washington County,FL,12133,12,25473,2021-01-21,2315,20.0,12.7,14.1,16.0
1150358,Washington County,FL,12133,12,25473,2021-01-22,2326,11.0,11.7,13.4,16.4
1153500,Washington County,FL,12133,12,25473,2021-01-23,2345,19.0,12.0,13.4,16.0


# Add %Diff vs yesterday, 7 days ago, 14 days ago, 28 days ago

In [15]:
# Naming pct_dt_<n>_<varname>
#   pct => Percentage
#   dt => Difference
#   n is # of days difference ie 1 => vs yesterday, 7 => vs 7 days ago
#   varname => Variable Name We are differencing

df_cases_7day_tidy['pct_dt_7_NewCases'] = (df_cases_7day_tidy['NewCases'] - df_cases_7day_tidy['NewCases'].shift(7)) / df_cases_7day_tidy['NewCases'].shift(7)
df_cases_7day_tidy['pct_dt_7_7day_avgs'] = (df_cases_7day_tidy['7day_avg'] - df_cases_7day_tidy['7day_avg'].shift(7)) / df_cases_7day_tidy['7day_avg'].shift(7)

In [16]:
df_cases_7day_tidy.tail(10)

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,pct_dt_7_NewCases,pct_dt_7_7day_avgs
1125222,Washington County,FL,12133,12,25473,2021-01-14,2226,29.0,15.4,18.2,17.2,-0.093750,-0.266667
1128364,Washington County,FL,12133,12,25473,2021-01-15,2244,18.0,15.1,19.5,17.4,-0.100000,-0.368201
1131506,Washington County,FL,12133,12,25473,2021-01-16,2261,17.0,14.7,17.4,17.4,-0.150000,-0.268657
1134648,Washington County,FL,12133,12,25473,2021-01-17,2263,2.0,14.6,16.6,17.3,-0.333333,-0.219251
1137790,Washington County,FL,12133,12,25473,2021-01-18,2273,10.0,15.3,16.1,17.0,1.000000,-0.094675
1140932,Washington County,FL,12133,12,25473,2021-01-19,2282,9.0,14.7,16.1,17.0,-0.307692,-0.155172
1144074,Washington County,FL,12133,12,25473,2021-01-20,2295,13.0,14.0,14.9,16.6,-0.277778,-0.119497
1147216,Washington County,FL,12133,12,25473,2021-01-21,2315,20.0,12.7,14.1,16.0,-0.310345,-0.175325
1150358,Washington County,FL,12133,12,25473,2021-01-22,2326,11.0,11.7,13.4,16.4,-0.388889,-0.225166
1153500,Washington County,FL,12133,12,25473,2021-01-23,2345,19.0,12.0,13.4,16.0,0.117647,-0.183673


In [50]:

source = df_cases_7day_tidy[df_cases_7day_tidy['Date'] == df_cases_7day_tidy['Date'].max()]
source.sort_values(by='pct_dt_7_7day_avgs', ascending=False, inplace=True)
source.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,pct_dt_7_NewCases,pct_dt_7_7day_avgs
1153436,Bay County,FL,12005,12,174705,2021-01-23,15159,115.0,172.9,145.4,137.8,0.982759,0.465254
1153472,Madison County,FL,12079,12,18493,2021-01-23,1770,14.0,10.4,9.4,9.3,0.076923,0.253012
1153469,Leon County,FL,12073,12,293582,2021-01-23,24991,253.0,228.1,215.8,232.2,0.571429,0.121436
1153465,Jefferson County,FL,12065,12,14246,2021-01-23,1241,8.0,9.3,9.3,10.6,0.333333,0.000000
1153473,Manatee County,FL,12081,12,403253,2021-01-23,27224,234.0,191.9,192.4,212.4,0.695652,-0.005699


In [63]:
date_time_str = '2020-10-01'
date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d')

field = 'pct_dt_7_7day_avgs'
field_title = "Pct Chg in 7dayAvg since 7 days ago"
for countyFIPS in source['countyFIPS'].head(5):
    chart_title = str(countyFIPS)
    state_df = df_cases_7day_tidy[(df_cases_7day_tidy['countyFIPS'] == countyFIPS) & (df_cases_7day_tidy['Date'] >= date_time_obj )]
    chart1 = alt.Chart(state_df).mark_bar().encode(
                         x=alt.X('yearmonthdate(Date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                         y=alt.Y(field, title=field_title),
                         ).properties( width=400, height=400, title=chart_title)
    display(chart1)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [39]:
date_time_str = '2020-10-01'

date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d')

# source = data.disasters.url
source = df_cases_7day_tidy[df_cases_7day_tidy['Date'] >= date_time_obj ][0:2000]
alt.Chart(source).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('Date:O', axis=alt.Axis(labelAngle=0)),
    alt.Y('countyFIPS:N'),
    alt.Size('pct_dt_7_NewCases:Q',
#         scale=alt.Scale(range=[0, 100]),
        legend=alt.Legend(title='Annual Global Deaths')
    ),
    alt.Color('countyFIPS:N', legend=None)
).properties(
    width=450,
    height=400
)

alt.Chart(...)

In [21]:
import altair as alt
from vega_datasets import data

source = data.disasters.url

alt.Chart(source).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('Year:O', axis=alt.Axis(labelAngle=0)),
    alt.Y('Entity:N'),
    alt.Size('Deaths:Q',
        scale=alt.Scale(range=[0, 4000]),
        legend=alt.Legend(title='Annual Global Deaths')
    ),
    alt.Color('Entity:N', legend=None)
).properties(
    width=450,
    height=320
).transform_filter(
    alt.datum.Entity != 'All natural disasters'
)

alt.Chart(...)

# BELOW HERE is BROKEN

In [ ]:
# Get per person counts
pd.set_option('display.float_format', lambda x: '%.2f' % x)

pd.set_option('mode.chained_assignment', None)
a=df_cases_7day_tidy[df_cases_7day_tidy['Date'] == df_cases_7day_tidy['Date'].max()]

# a['7day_ago'] = a['7day_avg'] - a['7day_avg'].shift(7)
a['7day_avg/1000'] = a['7day_avg'] / a['population'] * 1000
a['14day_avg/1000'] = a['14day_avg'] / a['population'] * 1000
a['28day_avg/1000'] = a['28day_avg'] / a['population'] * 1000

pd.set_option('mode.chained_assignment', 'warn')
a.head(10)

In [ ]:
# Get Diffs
a=df_cases_7day_tidy[df_cases_7day_tidy['Date'] == df_cases_7day_tidy['Date'].max()]
diff_days = [7, 14, 28]
for ddays in diff_days:
    new_col_label = 'diff vs previous' + str(ddays) + ' days'
    col_label = str(ddays) + 'day_avg'
    a[new_col_label] = a[col_label] - a[col_label].shift(7)
  

pd.set_option('mode.chained_assignment', 'warn')
a.head(10)
# a['7day_ago/1000'] = a['7day_avg'] / a['population'] * 1000
# a['diff prev7'] =  a['7day']  - a['7day_ago']
# a['diff prev7/1000'] = a['diff prev7'] / a['population'] * 1000

# sort_by='diff prev7/1000'
# a.sort_values(by=sort_by, ascending=False, inplace=True)

In [ ]:

date_time_str = '2020-10-01'

date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d')

for countyFIPS in a['countyFIPS'].head(10):
    fips_df = df_cases_7day_tidy[(df_cases_7day_tidy['countyFIPS'] == countyFIPS) & (df_cases_7day_tidy['Date'] >= date_time_obj)]
    field = '7day'
    field_title = 'NewCases 7 Day Average'
    chart_title = a.iloc[0]['State'] + ': ' + a.iloc[0]['County Name']
    chart1 = alt.Chart(fips_df).mark_bar().encode(
                         x=alt.X('yearmonthdate(Date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                         y=alt.Y(field, title=field_title),
                         ).properties( width=600, height=400, title=chart_title)
    display(chart1)

In [ ]:
df_cases_7day_tidy.dtypes

In [ ]:
a.iloc[0]['State'] + ': '' + a.iloc[0]['County Name'] 
# a.columns

# PLAY AREA

In [ ]:
fips_df

In [ ]:
field = 'New Cases'
chart_title = 'Cas'
chart1 = alt.Chart(fips_df).mark_bar().encode(
                     x=alt.X('yearmonthdate(Date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y=alt.Y(field, title=field_title),
                     ).properties( width=400, height=400, title=chart_title)
chart1

In [ ]:
chart1

In [ ]:
d233 = pd.read_csv('County-Data/usafacts_cases_tidy.csv')